In [ ]:
import pickle

def printv(*args, **kwargs):
    global verbose
    if verbose:
        print(*args, **kwargs)

class TestCase():
    def __init__(self, fname, record):
        self.values = None
        self.fname = None
        if not record:
            with open(fname, 'rb') as f:
                self.values = iter(pickle.load(f))
        else:
            self.fname = fname
        self.record_ = record
        self.to_record = []
    
    def assert_eq(self, thing):
        if not self.record_:
            val = next(self.values)
            assert thing == val, (thing, val)
        else:
            self.to_record.append(thing)
            printv(thing)
    
    def assert_is(self, thing):
        if not self.record_:
            val = next(self.values)
            assert thing is val, (thing, val)
        else:
            self.to_record.append(thing)
            printv(thing)
    
    def record(self):
        if self.record_:
            with open(self.fname, 'wb') as f:
                pickle.dump(self.to_record, f)

In [ ]:
from IPython import get_ipython
%run forestry.ipynb

time.sleep(0.000001)
cli.execute_command('q')

In [ ]:
pf = Princess(Genes(
        (BeeSpecies.FOREST, BeeSpecies.FOREST),
        (BeeFertility(2), BeeFertility(2))
     ))
dm = Drone(Genes(
        (BeeSpecies.MEADOWS, BeeSpecies.MEADOWS),
        (BeeFertility(2), BeeFertility(2))
))

In [ ]:
random.seed(2)
verbose = True

printv(pf, dm)
qc = pf.mate(dm)
pc, dc, dc2 = qc.die()
_, dc, dc2 = qc.die()
pc.inspect()
dc.inspect()
printv(pc)
printv(dc)

qc = pc.mate(dc)
pc, dc, dc2 = qc.die()
for i in range(3):
    _, _, dc = qc.die()
pc.inspect()
dc.inspect()
printv(pc)
printv(dc)
assert pc.genes == Genes(species=(BeeSpecies.COMMON, BeeSpecies.COMMON), fertility=(BeeFertility(2), BeeFertility(2)))

random.seed(40)
qcu = pc.mate(dm)
pcu, dcu, dcu2 = qcu.die()
pcu.inspect()
dcu.inspect()
qcu = pcu.mate(dcu)
pcu, _, dcu = qcu.die()
pcu.inspect()
dcu.inspect()
printv(pcu)
printv(dcu)
assert pcu.genes == Genes(species=(BeeSpecies.CULTIVATED, BeeSpecies.CULTIVATED), fertility=(BeeFertility(2), BeeFertility(2)))

random.seed(57)
qn = pcu.mate(dc)
pn, dn, _ = qn.die()
pn.inspect()
dn.inspect()
printv(pn)
printv(dn)

random.seed(17)
qn = pn.mate(dn)
pn, _, dn = qn.die()
pn.inspect()
dn.inspect()
printv(pn)
printv(dn)
assert pn.genes == Genes(species=(BeeSpecies.NOBLE, BeeSpecies.NOBLE), fertility=(BeeFertility(2), BeeFertility(2)))

In [ ]:
from collections import Counter
l = []
for i in range(10000):
    _, dc, dc2 = pf.mate(dm).die()
    dc.inspect()
    dc2.inspect()
    l.append(dc)
    l.append(dc2)
c = Counter(l)
s = sum(c.values())
for key in c:
    print(*key.genes.species, c[key] / s)

In [ ]:
from collections import Counter
l = []
princess = Princess(Genes(
        (BeeSpecies.COMMON, BeeSpecies.CULTIVATED),
        (BeeFertility(2), BeeFertility(2))
     ))
drone = Drone(Genes(
        (BeeSpecies.MEADOWS, BeeSpecies.MEADOWS),
        (BeeFertility(2), BeeFertility(2))
))
for i in range(10000):
    _, dc, dc2 = princess.mate(drone).die()
    dc.inspect()
    dc2.inspect()
    l.append(dc)
    l.append(dc2)
c = Counter(l)
s = sum(c.values())
for key in c:
    print(*key.genes.species, c[key] / s)

In [ ]:
inv = Inventory(100, render=printv)
inv.place_bees([pf, dm])
inv.render()
inv.mate(0, 1)
inv.render()
random.seed(0)
inv.place_bees(inv[0].take().die())
inv.render()
assert inv.empty_slots() == 97, inv.empty_slots()
assert inv.take(0).genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
assert inv.take(1).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
assert inv.take(2).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))

In [ ]:
verbose = False
random.seed(0)
r = Resources(honey=0, render=printv)
api = Apiary(add_resources=r.add_resources, render=printv)
api.put_princess(pf)
assert api.princess.slot.genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
api.render()

api.put_drone(dm)
api.render()
assert isinstance(api.princess.slot, Queen)
assert api.princess.slot.genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))

api.update()
api.render()
r.render()
assert r['honey'] == 0, r['honey']


api.update()
api.update()
api.render()
assert r['honey'] == 1, r['honey']

api.update()
api.render()
r.render()
assert api.inv.empty_slots() == 4, inv.empty_slots()
assert api.inv.take(0).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
assert api.inv.take(1).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
assert api.inv.take(2).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))

In [ ]:
verbose = True
random.seed(0)
r = Resources(honey=0, render=printv)
api = Apiary(add_resources=r.add_resources, render=printv)

steps = 10000
for i in range(steps):
    api.put(pf)
    api.put(dm)
    api.update()
    api.update()
    api.update()
    api.update()
    api.inv.take(0)
    api.inv.take(1)
    api.inv.take(2)
r.render()
for res in ['honey', 'wood']:
    discrepancy = abs(r[res] / 3 / steps - 0.5)
    printv(res, discrepancy)
    assert discrepancy < 0.1

In [ ]:
try:
    verbose = True
    tc = TestCase('cli.test', record=False)
    random.seed(0)

    sleep_time = 0.5

    cli = CLI()
    tc.assert_eq(cli.resources['honey'])
    tc.assert_eq(len(cli.apiaries))
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_eq(cli.out.outputs)

    cli.execute_command('unshow')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)

    cli.execute_command('unshow')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)

    cli.execute_command('unshow')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)

    cli.execute_command('inv')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    cli.execute_command('unshow')
    cli.execute_command('unshow')

    cli.execute_command('inv 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    cli.execute_command('unshow')
    cli.execute_command('unshow')

    cli.execute_command('apiary 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    cli.execute_command('unshow')
    cli.execute_command('unshow')

    cli.execute_command('show resources')
    cli.execute_command('show inv')
    cli.execute_command('show apiary 0')
    cli.execute_command('forage')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    tc.assert_eq(cli.inv[0].slot.genes)
    tc.assert_eq(cli.inv[1].slot.genes)
    tc.assert_eq(cli.inv.empty_slots())

    cli.execute_command('put 0 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    tc.assert_is(cli.inv[0].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)

    cli.execute_command('put 0 1')
    time.sleep(sleep_time)
    time.sleep(0.5) # UGLY: this sleep should help a little with unstable updates
    tc.assert_eq(cli.out.outputs)
    tc.assert_is(cli.inv[1].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_is(isinstance(cli.apiaries[0].princess.slot, Queen))
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs)
    tc.assert_eq(cli.apiaries[0].inv[0].slot.genes)
    tc.assert_eq(cli.apiaries[0].inv[1].slot.genes)
    tc.assert_eq(cli.apiaries[0].inv[2].slot.genes)

    cli.execute_command('reput 0 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    tc.assert_is(cli.inv[0].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)
    
    cli.execute_command('reput 0 1')
    time.sleep(sleep_time)
    time.sleep(0.5) # UGLY: this sleep should help a little with unstable updates
    tc.assert_eq(cli.out.outputs)
    tc.assert_is(cli.inv[1].is_empty())
    tc.assert_eq(cli.inv.empty_slots())
    tc.assert_is(isinstance(cli.apiaries[0].princess.slot, Queen))
    tc.assert_eq(cli.apiaries[0].princess.slot.genes)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs)
    time.sleep(1)
    tc.assert_eq(cli.out.outputs)
    tc.assert_eq(cli.apiaries[0].inv[0].slot.genes)
    tc.assert_eq(cli.apiaries[0].inv[1].slot.genes)
    tc.assert_eq(cli.apiaries[0].inv[2].slot.genes)
    
    cli.execute_command('reput 0 0')
    cli.execute_command('reput 0 1')
    time.sleep(4)

    cli.execute_command('reput 0 0')
    cli.execute_command('reput 0 1')
    time.sleep(5)
    tc.assert_eq(cli.resources['honey'])
    
    cli.execute_command('take 0 0')
    cli.execute_command('take 0 1')
    cli.execute_command('show inv 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    
    cli.execute_command('inspect 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)

    cli.execute_command('throw 0')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)

    cli.execute_command('swap 0 1')
    cli.execute_command('unshow')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)

    tc.record()
    print('wow, all tests passed')
    
finally:
    cli.execute_command('q')

In [ ]:
try:
    verbose = True
    tc = TestCase('cli.build.test', record=False)
    random.seed(4)

    sleep_time = 0.5

    cli = CLI()
    tc.assert_eq(cli.out.outputs)
    cli.resources.add_resources({'wood': 4, 'flowers': 4, 'honey': 9})
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    cli.execute_command('forage')
    cli.execute_command('forage')
    cli.execute_command('put 0 0')
    cli.execute_command('put 0 1')
    time.sleep(5)
    tc.assert_eq(cli.out.outputs)

    
    cli.execute_command('take 0 0')
    cli.execute_command('take 0 1')
    cli.execute_command('take 0 2')
    cli.execute_command('put 0 2')
    cli.execute_command('put 0 3')
    time.sleep(5)
    tc.assert_eq(cli.out.outputs)
    
    cli.execute_command('build apiary')
    cli.execute_command('show a 1')
    time.sleep(sleep_time)
    tc.assert_eq(cli.out.outputs)
    
    cli.execute_command('put 1 0')
    cli.execute_command('put 1 1')
    time.sleep(5)
    tc.assert_eq(cli.out.outputs)
    
    time.sleep(sleep_time)
    
    tc.record()
    print('wow, all tests passed')
finally:
    cli.execute_command('q')